# L03: RAG

During this lab, you will connect a fine-tuned language model with an external knowledge base (a PDF document)
Goal: Model should answer your questions based on your own data

Step 1. Environment

In [ ]:
!pip install -q faiss-cpu sentence-transformers transformers PyPDF2
import torch
import numpy as np
import textwrap
import faiss
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.0 MB/s eta 0:00:00


Step 2. Load the Model and Tokenizer

As discussed, the model for today's lab is Qwen3-4B-Instruct-2507

It is a fine-tuned model specialized in following instructions.

Go to HuggingFace and copy-paste the model and tokenizer: https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507


In [ ]:
model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Step 3. External Knowledge Base

Load the PDF file and extract the text

In [ ]:
pdf = "procedure.pdf"
reader = PdfReader(pdf)
text = ""
for page in reader.pages:
    page_text = page.extract_text()
    if page_text:
        text += page_text

Step 4.Preprocessing the document

In [ ]:
def chunk_text(text, chunk_size, overlap):
  words = text.split()
  chunks = []
  start = 0

  while start < len(words):
    end = start + chunk_size
    chunk = ' '.join(words[start:end])
    chunks.append(chunk)
    start += chunk_size - overlap

  return chunks

chunks = chunk_text(text, 400, 50)

Step 5.Embedding and Vector Database

Embedding model = all-MiniLM-L6-v2

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")
embeddings = embedder.encode(chunks, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step 6. Retriever

In [ ]:
def retrieve_context(query, k=3):
  query_vec = embedder.encode([query], convert_to_numpy=True)
  distances, indices = index.search(np.array(query_vec), k)
  retrieved = []

  for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
    retrieved.append({'chunk_id' : int(idx), 'text' : chunks[idx], 'distance' : dist})

  return retrieved

Step 7. Generate Answer

You can use this:

prompt = f"""

    Context:
    {context}

    Question:
    {query}

    Answer:"""

In [ ]:
def generate_answer(query, context_chunks):
  context = "\n\n".join([c['text'] for c in context_chunks])

  prompt = f"""

  Tell me in a dramatic tone, what happens when you dont wash your hands properly. Use bullet points, emojis etc.

  Context:
  {context}

  Question:
  {query}

  Answer:"""

  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
  answer = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
  return answer.strip()

Step 8. RAG pipeline

In [ ]:
def rag_query(query):
  context_chunks = retrieve_context(query, k=3)
  answer = generate_answer(query, context_chunks)
  return {'query': query, 'answer': answer, 'sources' : context_chunks}

RESULTS....

In [ ]:
query = ""
result = rag_query(query)
print(result["answer"])

🚨 **DANGER!** 🚨  
  🤯 What happens when you DON’T wash your hands properly? 🤯  

  ⚠️ **BUT WAIT — LET’S BREAK IT DOWN LIKE A DRAMATIC MOVIE SCENE!** ⚠️  

---

🎬 **SCENE: THE HALL OF INFECTION** 🎬  
📍 *A dimly lit hospital corridor. The air is thick with tension. A single drop of blood falls from a surgical glove. The lights flicker. A whisper echoes...*  

---

🔥 **1. THE FIRST SNEAKY ATTACK – MICROBES INVADE!** 🔥  
🦠 *Your hands are a
